# 107 - Minimal Network

## Problem Statement

The following undirected network consists of seven vertices and twelve edges with a total weight of 243.

<div class="center">
<img src="images/0107_1.png" style="background-color: white;" class="dark_img" alt=""><br></div>

The same network can be represented by the matrix below.

<table cellpadding="5" cellspacing="0" border="1"><tr><td>    </td><td><b>A</b></td><td><b>B</b></td><td><b>C</b></td><td><b>D</b></td><td><b>E</b></td><td><b>F</b></td><td><b>G</b></td>
</tr><tr><td><b>A</b></td><td>-</td><td>16</td><td>12</td><td>21</td><td>-</td><td>-</td><td>-</td>
</tr><tr><td><b>B</b></td><td>16</td><td>-</td><td>-</td><td>17</td><td>20</td><td>-</td><td>-</td>
</tr><tr><td><b>C</b></td><td>12</td><td>-</td><td>-</td><td>28</td><td>-</td><td>31</td><td>-</td>
</tr><tr><td><b>D</b></td><td>21</td><td>17</td><td>28</td><td>-</td><td>18</td><td>19</td><td>23</td>
</tr><tr><td><b>E</b></td><td>-</td><td>20</td><td>-</td><td>18</td><td>-</td><td>-</td><td>11</td>
</tr><tr><td><b>F</b></td><td>-</td><td>-</td><td>31</td><td>19</td><td>-</td><td>-</td><td>27</td>
</tr><tr><td><b>G</b></td><td>-</td><td>-</td><td>-</td><td>23</td><td>11</td><td>27</td><td>-</td>
</tr></table>

However, it is possible to optimise the network by removing some edges and still ensure that all points on the network remain connected. The network which achieves the maximum saving is shown below. It has a weight of 93, representing a saving of 243 − 93 = 150 from the original network.

<div class="center">
<img src="images/0107_2.png" style="background-color: white;" class="dark_img" alt=""><br></div>

Using <a href="inputs/0107_network.txt">network.txt</a> (right click and 'Save Link/Target As...'), a 6K text file containing a network with forty vertices, and given in matrix form, find the maximum saving which can be achieved by removing redundant edges whilst ensuring that the network remains connected.

## Solution

This is a classic problem. We are asked to find a Minimum Spanning Tree (MST) of the network. Here we use Kruskal's algorithm. First all the edges are sorted based on their weight. Then, the idea of the algorithm is to greedily add the edge with lowest associated weight that does not for a cycle in the the already selected vertices. To do this efficiently, we use a disjoint set (union find) data structure. As soon at all the vertices are connected, we stop adding edges.

Here, we are looking for the total saving which is half the sum (because of the symmetry of the matrix) of all the entries in the adjacency matrix minus the total weight of the MST.

In [2]:
import numpy as np

# Path to your text file
file_path = 'inputs/0107_network.txt'

def read_and_convert_matrix(file_path):
    # Initialize an empty list to hold each row of the matrix
    matrix = []

    with open(file_path, 'r') as file:
        for line in file:
            # Split line into elements, replace '-' with 'nan' (Not a Number)
            row = [np.nan if x == '-' else int(x) for x in line.strip().split(',')]
            # Append the row to the matrix
            matrix.append(row)

    # Convert the list to a NumPy array for better handling of numerical data
    return np.array(matrix, dtype=float)

# Call the function and print the resulting matrix
matrix = read_and_convert_matrix(file_path)

In [3]:
class UnionFind:

    def __init__(self, n):
        self.root = [i for i in range(n)]
        self.rank = [1] * n
        self.n_groups = n

    def find(self, x):
        if x == self.root[x]:
            return x
        self.root[x] = self.find(self.root[x])
        return self.root[x]

    def union(self, x, y):
        root_x = self.find(x)
        root_y = self.find(y)
        if root_x != root_y:
            if self.rank[root_x] > self.rank[root_y]:
                self.root[root_y] = root_x
            elif self.rank[root_y] > self.rank[root_x]:
                self.root[root_x] = self.root[root_y]
            else:
                self.root[root_y] = root_x
                self.rank[root_x] += 1
            self.n_groups -= 1

    def is_connected(self, x, y):
        return self.find(x) == self.find(y)



# Generate the list of edges and weight from the matrix
edges = []
for i in range(len(matrix)):
    for j in range(i):
        if not np.isnan(matrix[i, j]):
            edges.append((i, j, matrix[i, j]))
edges = sorted(edges, key=lambda x: x[2])  # Sort by weight

# Initialise total weight of the MST and UF data structure
total_weight = 0
uf = UnionFind(40)
for u, v, weight in edges:
    # If vertices u and v are not connected, the edge between u and v is part of the MST
    if not uf.is_connected(u, v):
        uf.union(u, v)
        total_weight += weight
        if uf.n_groups == 1:
            break

# Compute savings
np.nansum(matrix) // 2 - total_weight

259679.0